In [1]:
import os
import csv

In [2]:
ls

Modelfile*            demo/                     naive_RAG.ipynb*
PhoGPT-4B-Chat-gguf/  demo.py*                  nltk_data/
RAG_components.py*    demo_RAG.ipynb*           qa_data.csv*
RAG_test.ipynb*       demo_using_models.ipynb*  qa_data.json*
RAGapp.ipynb*         django-ui.ipynb*          resources/
__init__.py*          install.txt*              test-dir/
__pycache__/          legal-documents/          vinallama-legal-chat/
advanced_RAG.ipynb*   log/                      vistral-7b-legal-chat-final/
bge-m3/               main.ipynb*               vistral-legal-chat/
bge-small-en-v1.5/    modelfiles/               weights/
chat_engine/          my_venv/


In [3]:
os.chdir(r"chat_engine")

In [5]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.schema import QueryBundle
from llama_index.llms.gemini import Gemini

import json
import os

from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.postprocessor.cohere_rerank import CohereRerank

from llama_index.llms.ollama import Ollama
from llama_index.core.settings import Settings

from dotenv import load_dotenv

from prompts import gen_qa_prompt, gen_rag_answer
from RAG import *

In [4]:
from RAG import format_chatbot_output
from RAG import RAG

In [6]:
def extract_and_clean_column(file_path):
    """
    Reads the first column of a CSV file, cleans up line breaks, 
    and returns a list of strings from the rows, skipping the header.
    
    :param file_path: Path to the CSV file
    :return: List of cleaned strings from the first column
    """
    result = []

    # Open the CSV file
    with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        
        # Skip the header
        next(reader)
        
        # Iterate over the rows in the CSV
        for row in reader:
            if row:  # Make sure the row is not empty
                # Join lines in the cell and clean up extra spaces/newlines
                cleaned_text = ' '.join(row[0].split())
                result.append(cleaned_text)

    return result

In [7]:
# Example usage
file_path = 'answers-comparison-vistral.csv'
questions = extract_and_clean_column(file_path)


In [8]:
cleaned_questions = [q for q in questions if q.strip()]

In [15]:
print(len(cleaned_questions))

99


In [ ]:
for index,question in enumerate(cleaned_questions):
    index = index + 1
    print('index', index)
    print('question', question)
    print("===============================================================")
    print()

In [9]:
rag = RAG(
        model_name="vistral-legal-chat"    # vistral, phogpt, vinallama
    )

Deep Lake Dataset in hub://hunter/Vietnamese-law-RAG-semantic-chuning already exists, loading from the storage


In [15]:
# Prepare data for CSV
csv_data = [["Question", "Answer"]]  # Header row


In [ ]:
question = 'Đối với hợp đồng vay không kỳ hạn và không có lãi thì bên cho vay có quyền đòi lại tài sản và bên vay cũng có quyền trả nợ vào bất cứ lúc nào có đúng không?'
answer = rag.base_answer(question)
final_answer = format_chatbot_output(answer)
print(final_answer)

In [17]:
for question in cleaned_questions:
    answer = rag.base_answer(question)
    final_answer = format_chatbot_output(answer)
    csv_data.append([question, final_answer])

In [18]:


# Write data to CSV
with open('ver2_baseline_model_questions_and_answers.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)